In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Gooogle-Stock-Price-V2'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./train.csv')

In [3]:
test_data = pd.read_csv('./test.csv')

In [4]:
test_data

            Date    Open    High     Low   Close      Volume
0       1/3/2012  325.25  332.83  324.97  663.59   7,380,500
1       1/4/2012  331.27  333.87  329.08  666.45   5,749,400
2       1/5/2012  329.83  330.75  326.89  657.21   6,590,300
3       1/6/2012  328.34  328.77  323.68  648.24   5,405,900
4       1/9/2012  322.04  322.29  309.46  620.76  11,688,800
...          ...     ...     ...     ...     ...         ...
1253  12/23/2016  790.90  792.74  787.28  789.91     623,400
1254  12/27/2016  790.68  797.86  787.66  791.55     789,100
1255  12/28/2016  793.70  794.23  783.20  785.05   1,153,800
1256  12/29/2016  783.33  785.93  778.92  782.79     744,300
1257  12/30/2016  782.75  782.78  770.41  771.82   1,770,000

[1258 rows x 6 columns]

In [5]:
data

         Date    Open    High     Low   Close     Volume
0    1/3/2017  778.81  789.63  775.80  786.14  1,657,300
1    1/4/2017  788.36  791.34  783.16  786.90  1,073,000
2    1/5/2017  786.08  794.48  785.02  794.02  1,335,200
3    1/6/2017  795.26  807.90  792.20  806.15  1,640,200
4    1/9/2017  806.40  809.97  802.83  806.65  1,272,400
5   1/10/2017  807.86  809.13  803.51  804.79  1,176,800
6   1/11/2017  805.00  808.15  801.37  807.91  1,065,900
7   1/12/2017  807.14  807.39  799.17  806.36  1,353,100
8   1/13/2017  807.48  811.22  806.69  807.88  1,099,200
9   1/17/2017  807.08  807.14  800.37  804.61  1,362,100
10  1/18/2017  805.81  806.21  800.99  806.07  1,294,400
11  1/19/2017  805.12  809.48  801.80  802.17    919,300
12  1/20/2017  806.91  806.91  801.69  805.02  1,670,000
13  1/23/2017  807.25  820.87  803.74  819.31  1,963,600
14  1/24/2017  822.30  825.90  817.82  823.87  1,474,000
15  1/25/2017  829.62  835.77  825.06  835.67  1,494,500
16  1/26/2017  837.81  838.00  

In [6]:
data.to_csv('./Cleaned-Data.csv')
data.to_json('./cleaned-data.json')

In [7]:
test_data.to_csv('./Cleaned-test_data.csv')
test_data.to_json('./cleaned-test_data.json')

In [8]:
data = torch.from_numpy(np.array(data['Open'].tolist())).to(device).float().view(1,-1)
test_data = torch.from_numpy(np.array(test_data['Open'].tolist())).to(device).float().view(1,-1)

In [9]:
data_input = data[:1,:-1]

In [10]:
data_target = data[:1,1:]

In [11]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 128
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.lstm3 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t3,c_t3 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [12]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 250

In [13]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 1000
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        preds = preds[0].view(-1).cpu().detach().numpy()
        wandb.log({'Val Loss':loss.item()})
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy(),'r')
        plt.plot(np.arange(n,n+future),preds[n:],'b','-')
        plt.plot(np.arange(n,n+future),test_data[n:].view(-1).cpu().detach().numpy(),'g','-')
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

In [14]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 128
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.lstm3 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t3,c_t3 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.linear(h_t3)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [15]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 250

In [16]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 1000
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        preds = preds[0].view(-1).cpu().detach().numpy()
        wandb.log({'Val Loss':loss.item()})
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy(),'r')
        plt.plot(np.arange(n,n+future),preds[n:],'b','-')
        plt.plot(np.arange(n,n+future),test_data[n:].view(-1).cpu().detach().numpy(),'g','-')
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()